In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
movable = np.array(
        [[1, 0, 0, 0, 0, 0, 1],
         [1, 0, 0, 0, 0, 0, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 1, 1, 1]]
    ).astype(np.bool)

rewards = np.array(
        [[ 100,    0,    0,    0,    0,    0, -100],
         [  -1,    0,    0,    0,    0,    0,   -1],
         [  -1,   -1,   -1,   -1,   -1,   -1,   -1],
         [   0,    0,    0,   -1,    0,    0,    0],
         [   0,    0,    0,   -1,    0,    0,    0],
         [   0,    0,    0,   -1,    0,    0,    0],
         [   0,    0,    0,   -1,   -1,   -1,   -1]]
    ).astype(np.float)

pos = np.array([3, 3])
actions = np.array([[0, 0], [0, -1], [0, 1], [-1, 0], [1, 0]])
gamma = 0.8

In [ ]:
def calc_new_value(pos, value):
    return gamma * np.max(
        [
            rewards[tuple(pos + a)] + value[tuple(pos + a)]
                for a in actions
                if np.all(pos + a >= 0)
                    and np.all(pos + a < 7)
                    and movable[tuple(pos + a)]
                    and movable[tuple(pos)]
        ], initial=np.NINF)

In [ ]:
value = np.zeros((7, 7))
for _ in range(20):
    new_value = np.zeros_like(value)
    for pos in np.ndindex(value.shape):
        new_value[pos] = calc_new_value(pos, value)
    value = new_value
    plt.imshow(value)
    plt.show()